# Session 12 - Caching, LRU Cache, and Smart Memory Systems

_Exercise notebook (with solutions)._

## Exercise 1: Manual Cache (Dictionary)

- Implement a manual cache using a global/local dictionary.
- Write a function `heavy_calculation(x)` that:
  - returns from cache if value exists;
  - otherwise computes (simulates expensive work), stores, and returns it.
- Print different messages for "from cache" vs "new computation".
- Test with repeated calls for the same `x`.

In [5]:
import time

manual_cache = {}


def heavy_calculation(x: int) -> int:
    if x in manual_cache:
        print(f"[cache hit] x={x}")
        return manual_cache[x]

    print(f"[new compute] x={x}")
    time.sleep(0.5)  # simulate expensive work
    result = x * x + 42
    manual_cache[x] = result
    return result


for value in [5, 7, 5, 7, 9, 5]:
    print(f"result({value}) = {heavy_calculation(value)}")


[new compute] x=5
result(5) = 67
[new compute] x=7
result(7) = 91
[cache hit] x=5
result(5) = 67
[cache hit] x=7
result(7) = 91
[new compute] x=9
result(9) = 123
[cache hit] x=5
result(5) = 67


## Exercise 2: LRU Cache with `functools.lru_cache`

- Use `@lru_cache(maxsize=3)` on a slow function (for example with `time.sleep`).
- Call the function on a value sequence so you can observe:
  - cache hits;
  - eviction of the least recently used entry once `maxsize` is exceeded.
- Show when the function computes vs serves from cache.

In [6]:
import time
from functools import lru_cache


@lru_cache(maxsize=3)
def slow_square(x: int) -> int:
    print(f"[computed] slow_square({x})")
    time.sleep(0.3)
    return x * x


for x in [1, 2, 3, 1, 4, 2, 5, 1]:
    before = slow_square.cache_info()
    result = slow_square(x)
    after = slow_square.cache_info()
    served_from_cache = after.hits > before.hits
    source = "cache" if served_from_cache else "computed"
    print(f"x={x:>2} -> {result:>2} [{source}]")

print("cache_info:", slow_square.cache_info())


[computed] slow_square(1)
x= 1 ->  1 [computed]
[computed] slow_square(2)
x= 2 ->  4 [computed]
[computed] slow_square(3)
x= 3 ->  9 [computed]
x= 1 ->  1 [cache]
[computed] slow_square(4)
x= 4 -> 16 [computed]
[computed] slow_square(2)
x= 2 ->  4 [computed]
[computed] slow_square(5)
x= 5 -> 25 [computed]
[computed] slow_square(1)
x= 1 ->  1 [computed]
cache_info: CacheInfo(hits=1, misses=7, maxsize=3, currsize=3)


## Exercise 3: Custom LRU Cache (with `OrderedDict`)

- Create class `LRUCache(capacity)` that stores key -> value pairs.
- Implement methods:
  - `get(key)` -> returns value or `-1` if not found; mark key as recently used.
  - `set(key, value)` -> insert/update; if capacity exceeded, evict least recently used key.
- Write a short test scenario that demonstrates correct eviction.

In [7]:
from collections import OrderedDict


class LRUCache:
    def __init__(self, capacity: int):
        self.capacity = capacity
        self.data = OrderedDict()

    def get(self, key):
        if key not in self.data:
            return -1
        self.data.move_to_end(key)  # mark as recently used
        return self.data[key]

    def set(self, key, value):
        if key in self.data:
            self.data[key] = value
            self.data.move_to_end(key)
            return

        self.data[key] = value
        if len(self.data) > self.capacity:
            evicted_key, evicted_value = self.data.popitem(last=False)
            print(f"Evicted: {evicted_key} -> {evicted_value}")

    def __repr__(self):
        return f"LRUCache(capacity={self.capacity}, data={list(self.data.items())})"


cache = LRUCache(capacity=3)
cache.set("A", 10)
cache.set("B", 20)
cache.set("C", 30)
print(cache)
print("get('A') =", cache.get("A"))  # A becomes most recent
cache.set("D", 40)  # should evict B
print(cache)
print("get('B') =", cache.get("B"))


LRUCache(capacity=3, data=[('A', 10), ('B', 20), ('C', 30)])
get('A') = 10
Evicted: B -> 20
LRUCache(capacity=3, data=[('C', 30), ('A', 10), ('D', 40)])
get('B') = -1


## Exercise 4: Expiring Cache for a Simulated API

- Implement `get_price(crypto)` with cache entries expiring after 10 seconds.
- Cache should store both value and expiration timestamp.
- If entry is valid, return from cache; otherwise recompute and overwrite.
- Test with `time.sleep` to observe behavior before and after expiration.

In [8]:
import random
import time

TTL_SECONDS = 10
price_cache = {}


def get_price(crypto: str) -> float:
    now = time.time()
    cached = price_cache.get(crypto)

    if cached is not None:
        value, expires_at = cached
        if now < expires_at:
            print(f"[cache hit] {crypto} -> {value:.2f}")
            return value

    # simulate API call
    value = round(random.uniform(10000, 70000), 2)
    expires_at = now + TTL_SECONDS
    price_cache[crypto] = (value, expires_at)
    print(f"[new fetch] {crypto} -> {value:.2f}")
    return value


print(get_price("BTC"))
print(get_price("BTC"))
print("waiting 10.5 seconds for expiration...")
time.sleep(10.5)
print(get_price("BTC"))


[new fetch] BTC -> 66397.47
66397.47
[cache hit] BTC -> 66397.47
66397.47
waiting 10.5 seconds for expiration...
[new fetch] BTC -> 15981.55
15981.55


## Exercise 5: Custom Cache Decorator

- Write decorator `cache_decorator(f)` that stores function results for identical arguments.
- In `wrapper(*args)`, check if `args` exist in cache:
  - if yes, return cached value and print "From cache";
  - if not, compute, store, and return.
- Apply decorator to a simple function (for example `calculate(x, y)`), then test with repeated calls.

In [9]:
def cache_decorator(func):
    cache = {}

    def wrapper(*args, **kwargs):
        key = (args, tuple(sorted(kwargs.items())))
        if key in cache:
            print("From cache")
            return cache[key]

        print("Computed now")
        result = func(*args, **kwargs)
        cache[key] = result
        return result

    return wrapper


@cache_decorator
def calculate(x, y):
    return x * y + x - y


print(calculate(3, 4))
print(calculate(3, 4))
print(calculate(10, 2))
print(calculate(10, 2))


Computed now
11
From cache
11
Computed now
28
From cache
28


## Exercise 6: LRU Cache for Temperature Conversion (last 5 results)

- Create a temperature conversion function (for example `c_to_f(c)` or `f_to_c(f)`).
- Use an LRU mechanism that keeps **the last 5 results**.
- Demonstrate that when you exceed 5 entries, the least recently used one is evicted.
- You may choose `lru_cache(maxsize=5)` or your own implementation.

In [10]:
from functools import lru_cache


@lru_cache(maxsize=5)
def c_to_f(celsius: float) -> float:
    print(f"[computed] c_to_f({celsius})")
    return celsius * 9 / 5 + 32


for c in [0, 10, 20, 30, 40, 50, 10, 60, 20]:
    before = c_to_f.cache_info()
    value = c_to_f(c)
    after = c_to_f.cache_info()
    source = "cache" if after.hits > before.hits else "computed"
    print(f"{c:>2}C -> {value:>5.1f}F [{source}]")

print("cache_info:", c_to_f.cache_info())


[computed] c_to_f(0)
 0C ->  32.0F [computed]
[computed] c_to_f(10)
10C ->  50.0F [computed]
[computed] c_to_f(20)
20C ->  68.0F [computed]
[computed] c_to_f(30)
30C ->  86.0F [computed]
[computed] c_to_f(40)
40C -> 104.0F [computed]
[computed] c_to_f(50)
50C -> 122.0F [computed]
10C ->  50.0F [cache]
[computed] c_to_f(60)
60C -> 140.0F [computed]
[computed] c_to_f(20)
20C ->  68.0F [computed]
cache_info: CacheInfo(hits=1, misses=8, maxsize=5, currsize=5)


## Exercise 7: Cached Factorials (decorated)

- Create function `factorial(n)`.
- Apply a cache decorator (standard or custom) so previously computed factorials are reused.
- Demonstrate reuse with repeated calls and by printing when computation happens vs cache usage.

In [11]:
from functools import lru_cache


@lru_cache(maxsize=None)
def factorial(n: int) -> int:
    if n < 0:
        raise ValueError("n must be >= 0")
    print(f"[computed] factorial({n})")
    if n <= 1:
        return 1
    return n * factorial(n - 1)


for n in [6, 6, 5, 7, 7]:
    before = factorial.cache_info()
    value = factorial(n)
    after = factorial.cache_info()
    source = "cache" if after.hits > before.hits else "computed"
    print(f"factorial({n}) = {value} [{source}]")

print("cache_info:", factorial.cache_info())


[computed] factorial(6)
[computed] factorial(5)
[computed] factorial(4)
[computed] factorial(3)
[computed] factorial(2)
[computed] factorial(1)
factorial(6) = 720 [computed]
factorial(6) = 720 [cache]
factorial(5) = 120 [cache]
[computed] factorial(7)
factorial(7) = 5040 [cache]
factorial(7) = 5040 [cache]
cache_info: CacheInfo(hits=4, misses=7, maxsize=None, currsize=7)


## Exercise 8: Caching System for API Data (real or simulated)

- Build a function that "queries" an API (real or simulated).
- Add caching with:
  - key derived from request parameters;
  - configurable expiration (TTL).
- Include a small demo showing reduced number of API calls thanks to cache.

In [12]:
import time

api_cache = {}
api_call_count = 0


def fake_api(endpoint: str, **params):
    global api_call_count
    api_call_count += 1
    time.sleep(0.25)  # simulate network latency
    return {
        "endpoint": endpoint,
        "params": params,
        "payload": f"data_for_{endpoint}_{params}",
        "call_number": api_call_count,
    }


def get_api_data(endpoint: str, ttl: int = 3, **params):
    key = (endpoint, tuple(sorted(params.items())))
    now = time.time()

    if key in api_cache:
        data, expires_at = api_cache[key]
        if now < expires_at:
            print("[cache hit]", key)
            return data

    print("[api call]", key)
    data = fake_api(endpoint, **params)
    api_cache[key] = (data, now + ttl)
    return data


print(get_api_data("weather", city="Cluj", unit="metric"))
print(get_api_data("weather", city="Cluj", unit="metric"))
print(get_api_data("weather", city="Iasi", unit="metric"))
print("waiting for TTL expiration...")
time.sleep(3.2)
print(get_api_data("weather", city="Cluj", unit="metric"))
print("Total API calls made:", api_call_count)


[api call] ('weather', (('city', 'Cluj'), ('unit', 'metric')))
{'endpoint': 'weather', 'params': {'city': 'Cluj', 'unit': 'metric'}, 'payload': "data_for_weather_{'city': 'Cluj', 'unit': 'metric'}", 'call_number': 1}
[cache hit] ('weather', (('city', 'Cluj'), ('unit', 'metric')))
{'endpoint': 'weather', 'params': {'city': 'Cluj', 'unit': 'metric'}, 'payload': "data_for_weather_{'city': 'Cluj', 'unit': 'metric'}", 'call_number': 1}
[api call] ('weather', (('city', 'Iasi'), ('unit', 'metric')))
{'endpoint': 'weather', 'params': {'city': 'Iasi', 'unit': 'metric'}, 'payload': "data_for_weather_{'city': 'Iasi', 'unit': 'metric'}", 'call_number': 2}
waiting for TTL expiration...
[api call] ('weather', (('city', 'Cluj'), ('unit', 'metric')))
{'endpoint': 'weather', 'params': {'city': 'Cluj', 'unit': 'metric'}, 'payload': "data_for_weather_{'city': 'Cluj', 'unit': 'metric'}", 'call_number': 3}
Total API calls made: 3


## Exercise 9: Execution Time Comparison - with and without cache

- Choose a sufficiently expensive function (simulated with `sleep` or heavy computation).
- Measure total time for a call set:
  - once without cache;
  - once with cache (manual / `lru_cache` / decorator).
- Print results and conclusion (observed time difference).

In [13]:
import time
from functools import lru_cache


def slow_work_no_cache(x: int) -> int:
    time.sleep(0.12)
    return x * x


@lru_cache(maxsize=None)
def slow_work_cached(x: int) -> int:
    time.sleep(0.12)
    return x * x


inputs = [1, 2, 3, 2, 1, 3, 4, 1, 2, 4, 3]

start = time.perf_counter()
out_no_cache = [slow_work_no_cache(x) for x in inputs]
time_no_cache = time.perf_counter() - start

start = time.perf_counter()
out_cached = [slow_work_cached(x) for x in inputs]
time_cached = time.perf_counter() - start

print("without cache:", round(time_no_cache, 4), "seconds")
print("with cache   :", round(time_cached, 4), "seconds")
print("same outputs :", out_no_cache == out_cached)

if time_cached < time_no_cache:
    speedup = time_no_cache / time_cached if time_cached > 0 else float("inf")
    print(f"Speedup: {speedup:.2f}x faster with cache")
else:
    print("No speedup observed in this run")


without cache: 1.321 seconds
with cache   : 0.4806 seconds
same outputs : True
Speedup: 2.75x faster with cache
